# Momentum in training neural network

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import nnfs
from nnfs.datasets import spiral_data

In [2]:
X,y = spiral_data(samples=100,classes=3)

In [3]:
# creating Dense Layer class  with backpropogation
class Dense:
    # layer initialization
    def __init__(self,n_inputs,n_neurons):
        super().__init__()
        self.weights =  0.01*np.random.randn(n_inputs,n_neurons)
        self.biases = np.zeros((1,n_neurons))
    
    # forward pass of Dense Layer
    def forward(self,inputs):
        # remember input values
        self.inputs = inputs
        # calculate output values from weights and inputs,weights,biases
        self.outputs = np.dot(inputs,self.weights) + self.biases
    
    # backward method or backpropogation 
    def backward(self,dvalues):
        # gradients on parameters
        # loss with respect to weights
        self.dweights = np.dot(self.inputs.T,dvalues)
        # loss with respect to biases
        self.dbiases = np.sum(dvalues,axis=0,keepdims=True)
        # loss with respect to the inputs
        self.dinputs = np.dot(dvalues,self.weights.T)

In [4]:
# creating relu class with backpropogation
class ReLU:
    # forward pass
    def forward(self,inputs):
        # remember input values
        self.inputs = inputs
        self.output = np.maximum(0,inputs)
    
    # backward method in relu activation function
    def backward(self,dvalues):
        # since we need to modify original variable,
        # lets make a copy of value event first
        self.dinputs = dvalues.copy()
        # Zero gradient where input values are negative
        self.dinputs[self.inputs <= 0] = 0

In [5]:
# creating softmax class
class Softmax:
    def forward(self,inputs):
        exp_values = np.exp(inputs - np.max(inputs,axis=1,keepdims=True))
        probabelities = exp_values/ np.sum(exp_values,axis=1,keepdims=True)
        self.output = probabelities

In [6]:
# Common loss class
class Loss:
    # Calculates the data and regularization losses
    # given model output and ground truth values
    def calculate(self, output, y):
        # Calculate sample losses
        sample_losses = self.forward(output, y)
        # Calculate mean loss
        data_loss = np.mean(sample_losses)
        # Return loss
        return data_loss

In [7]:
class CrossEntropyLoss(Loss):
    def forward(self,y_pred,y_true):
        # storing y_pred and y_true
        # self.y_true = y_true
        # self.y_pred = y_pred
        # number of samples in a batch
        samples = len(y_pred)

        # clip data to prevent division by 0
        # clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred,1e-7,1-1e-7)
        
        # probabelities for target values
        # only if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[
            range(samples),
            y_true]
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(
                y_pred_clipped*y_true,axis=1
            )
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods
    
    # backward pass or backpropogation
    def backward(self,dvalues,y_true):
        # number of samples
        samples = len(dvalues)
        # number of labels in every sample
        # we will use the first sample to count them
        labels = len(dvalues[0])
        # if labels are sparse turn them into one hot vector
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]
        # calculate gradient
        self.dinputs = -y_true / dvalues
        # normalize gradient
        self.dinputs = self.dinputs/samples

In [8]:
# classifier - Softmax Classifier
# Loss Function - Categorical Cross Entropy Loss Function
class Activation_Softmax_Loss_CategoricalCrossEntropy:
    # initializing class or class constructor
    def __init__(self):
        # setting activation function to softmax
        self.activation = Softmax()
        # setting loss function to categorical cross entropy
        self.loss = CrossEntropyLoss()
    
    # forward method of class
    def forward(self,inputs,y_true):
        # applying activation function to inputs
        self.activation.forward(inputs)
        # getting the output valus of softmax function to get input for loss function
        self.output = self.activation.output
        # applying loss function to inputs and y_true
        return self.loss.calculate(self.output,y_true)
    
    # backward pss
    def backward(self, d_values,y_true):
        # number of samples 
        samples = len(d_values)
        # if tables are one-hot encoded,
        # turn them into discrete values
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true,axis=1)
        # copying values of dvalues into input
        self.dinputs = d_values.copy()
        # calculate gradient
        self.dinputs[range(samples),y_true] -= 1
        # normalize gradient
        self.dinputs = self.dinputs/samples

In [9]:
# Gradient Descent Optimizer
class GradientDescent:
    # initialize optimizer - set settings
    # learning rate of 1. is default for this optimizer
    def __init__(self,learning_rate = 1,decay=0.,momentum = 0.):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iteration = 0
        self.momentum = momentum
    
    # call once before any parameter update
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1./(1.+self.decay*self.iteration))
    
    # update parameters
    def update_parameters(self,layer):
        # if we use momentum
        if self.momentum:
            # if layer does not contain momentum array create them
            # filled with zero
            if not hasattr(layer,'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)
            
            # build weights updates with momentum - take previous
            # updates multiplied by retain factor and update with current gradients
            weight_updates = self.momentum * layer.weight_momentums - self.current_learning_rate * layer.dweights
            layer.weight_momentums = weight_updates

            bias_updates = self.momentum * layer.bias_momentums - self.current_learning_rate * layer.dbiases
            layer.bias_momentums = bias_updates
        
        # vannila Gradient Descent (as before momentum update)
        else:
            weight_updates = -self.current_learning_rate * layer.dweights
            bias_updates = -self.current_learning_rate * layer.dbiases
        
        layer.weights += weight_updates
        layer.biases += bias_updates
    
    # call once after any parameter update
    def post_update_params(self):
        self.iteration += 1

In [10]:
# create dense layer with 2 inputs and 3 layers
dense_1 = Dense(2,64)
# create activation ReLU(to be used with dense layer)
relu = ReLU()
# create second dense layer with 64 input feature (as we take output of previous layer here ) and 3  output values(output values)
dense_2 = Dense(64,3)
# create softmax classifier's combined loss and activation
loss_activation = Activation_Softmax_Loss_CategoricalCrossEntropy()
# add a gradient descent optimizer
# 1e-3 = 1 * 10^-3
optimizer = GradientDescent(decay=1e-3,momentum=0.9)

# train in loop
for epoch in range(10001):
    # forward pass of our training data through this layer
    dense_1.forward(X)
    # perform a forward pass through activation function
    # take outpu of first dense layer here
    relu.forward(dense_1.outputs)
    # perform a forward passs through second dense layer
    # take output of relu layer
    dense_2.forward(relu.output)
    # perform a forward pass through activation function and loss function
    # takes output from second dense layer and return output here
    loss = loss_activation.forward(dense_2.outputs, y)

    # calculate accuracy from output of softmax activation and outputs
    # calculate values along first axis
    predictions = np.argmax(loss_activation.output,axis=1)

    if len(y.shape) == 2:
        y = np.argmax(y,axis=1)
    accuracy = np.mean(predictions == y)

    if not epoch % 100:
        print(f'epoch: {epoch}, acc: {accuracy:.3f}, loss: {loss:.3f}, Learning Rate: {optimizer.current_learning_rate}')
    
    # backward pass
    loss_activation.backward(loss_activation.output,y)
    dense_2.backward(loss_activation.dinputs)
    relu.backward(dense_2.dinputs)
    dense_1.backward(relu.dinputs)

    # update weights and biases
    optimizer.pre_update_params()
    optimizer.update_parameters(dense_1)
    optimizer.update_parameters(dense_2)
    optimizer.post_update_params()

epoch: 0, acc: 0.293, loss: 1.099, Learning Rate: 1
epoch: 100, acc: 0.463, loss: 1.029, Learning Rate: 0.9099181073703367
epoch: 200, acc: 0.413, loss: 1.028, Learning Rate: 0.8340283569641367
epoch: 300, acc: 0.633, loss: 0.796, Learning Rate: 0.7698229407236336
epoch: 400, acc: 0.670, loss: 0.726, Learning Rate: 0.7147962830593281
epoch: 500, acc: 0.783, loss: 0.554, Learning Rate: 0.66711140760507
epoch: 600, acc: 0.827, loss: 0.450, Learning Rate: 0.6253908692933083
epoch: 700, acc: 0.797, loss: 0.443, Learning Rate: 0.5885815185403178
epoch: 800, acc: 0.880, loss: 0.335, Learning Rate: 0.5558643690939411
epoch: 900, acc: 0.873, loss: 0.310, Learning Rate: 0.526592943654555
epoch: 1000, acc: 0.867, loss: 0.309, Learning Rate: 0.5002501250625312
epoch: 1100, acc: 0.890, loss: 0.261, Learning Rate: 0.4764173415912339
epoch: 1200, acc: 0.883, loss: 0.275, Learning Rate: 0.45475216007276037
epoch: 1300, acc: 0.890, loss: 0.279, Learning Rate: 0.43497172683775553
epoch: 1400, acc: 0.89